# Back Propagation

In order to understand backpropagation, you will implement a backpropagation algorithm for a neural network from scratch. <br/>
Backpropagation can be used for both classification and regression problems, we will focus on classification in
this exercise.

#### Exercise 1 : initialization of the network

In [1]:
from random import random, seed

In [2]:
def init_network(n_inputs, n_hidden, n_outputs): 
    network = list()

    # hidden layer 
    hidden_layer = [{'weights':[random() for i in range(n_inputs + 1)]} for i in range(n_hidden)]
    network.append(hidden_layer)

    # output layer 
    output_layer = [{'weights':[random() for i in range(n_hidden + 1)]} for i in range(n_outputs)]
    network.append(output_layer)

    return network


#### Exercise 2 : 

In [3]:
seed(1)
net = init_network(2,4,6)
for layer in net: 
    print(layer)

[{'weights': [0.13436424411240122, 0.8474337369372327, 0.763774618976614]}, {'weights': [0.2550690257394217, 0.49543508709194095, 0.4494910647887381]}, {'weights': [0.651592972722763, 0.7887233511355132, 0.0938595867742349]}, {'weights': [0.02834747652200631, 0.8357651039198697, 0.43276706790505337]}]
[{'weights': [0.762280082457942, 0.0021060533511106927, 0.4453871940548014, 0.7215400323407826, 0.22876222127045265]}, {'weights': [0.9452706955539223, 0.9014274576114836, 0.030589983033553536, 0.0254458609934608, 0.5414124727934966]}, {'weights': [0.9391491627785106, 0.38120423768821243, 0.21659939713061338, 0.4221165755827173, 0.029040787574867943]}, {'weights': [0.22169166627303505, 0.43788759365057206, 0.49581224138185065, 0.23308445025757263, 0.2308665415409843]}, {'weights': [0.2187810373376886, 0.4596034657377336, 0.28978161459048557, 0.021489705265908876, 0.8375779756625729]}, {'weights': [0.5564543226524334, 0.6422943629324456, 0.1859062658947177, 0.9925434121760651, 0.8599465287

### Forward Propagation

#### Exercise 3 : Neuron activation

In [4]:
# Weight is a network weight
# Input is an input (can be line of a dataset, should be a vector of the same size of the number of nodes in the network)
# Bias is a special weight that has no input to multiply with

# activation = sum(weight_i * input_i) + bias

In [5]:
# weights is a list of given weights
# inputs is a list of inputs

def activate(weights, inputs): 
    # To simplify, the bias = the last weight of weights
    activation = weights[-1]
    for i in range(len(weights) - 1): 
        activation += weights[i] * inputs[i]
    return activation

#### Exercise 4 : Neuron transfer

In [6]:
from math import exp

def transfer(activation): 
    return 1.0/(1.0 + exp(-activation))

#### Exercise 5 : Forward propagation

In [7]:
# Forward propagate input to a network output
def forward_propagate(network, row): 
    inputs = row
    for layer in network: 
        new_inputs = []
        for neuron in layer: 
            activation = activate(neuron['weights'], inputs)
            neuron['output'] = transfer(activation)
            new_inputs.append(neuron['output'])
        inputs = new_inputs
    return inputs

#### Exercise 6 : Testing our forward propagation

In [8]:
seed(1)
network = init_network(3, 2, 2)
row = [1, 0, 0]
output = forward_propagate(network, row)

output

[0.7138015773148616, 0.7020509238862008]

### Back Propagation Error

#### Exercise 7 : Transfer derivative

In [9]:
# if transfer function is sigmoid 
def transfer_derivative(output): 
    return output * (1.0 - output)

#### Exercise 8 : Error Backpropagation

In [10]:
# error = (expected - output) * transfer_derivative(output)

In [11]:
# Backpropagate error and store in neurons
def backward_propagate_error(network, expected): 
    for i in reversed(range(len(network))): 
        layer = network[i]
        errors = list()
        if i != len(network) - 1 : 
            for j in range(len(layer)): 
                error = 0.0
                for neuron in network[i+1]: 
                    error += (neuron['weights'][j] * neuron['delta'])
                errors.append(error)
        else: 
            for j in range(len(layer)): 
                neuron = layer[j]
                errors.append(expected[j] - neuron['output'])
        for j in range(len(layer)): 
            neuron = layer[j]
            neuron['delta'] = errors[j] * transfer_derivative(neuron['output'])

#### Exercise 9 : Test backward propagate error

In [12]:
expected = [0.7138015773148616, 0.7020509238862008]
backward_propagate_error(network, expected)

In [13]:
network

[[{'weights': [0.13436424411240122,
    0.8474337369372327,
    0.763774618976614,
    0.2550690257394217],
   'output': 0.5961462631100435,
   'delta': 0.0},
  {'weights': [0.49543508709194095,
    0.4494910647887381,
    0.651592972722763,
    0.7887233511355132],
   'output': 0.7831568031528053,
   'delta': 0.0}],
 [{'weights': [0.0938595867742349, 0.02834747652200631, 0.8357651039198697],
   'output': 0.7138015773148616,
   'delta': 0.0},
  {'weights': [0.43276706790505337, 0.762280082457942, 0.0021060533511106927],
   'output': 0.7020509238862008,
   'delta': 0.0}]]

#### Exercise 10 : Update weights

In [14]:
# weight = weight + learning rate * error * input

where **weight** is a given weight, <br/>
**learning rate** is a parameter that you must
specify, <br/>
**error** is the error calculated by the backpropagation procedure for the
neuron and <br/>
**input** is the input value that caused the error.

In [15]:
def update_weights(network, row, l_rate):
	for i in range(len(network)):
		inputs = row[:-1]
		if i != 0:
			inputs = [neuron['output'] for neuron in network[i - 1]]
		for neuron in network[i]:
			for j in range(len(inputs)):
				neuron['weights'][j] += l_rate * neuron['delta'] * inputs[j]
			neuron['weights'][-1] += l_rate * neuron['delta']

#### Exercise 11 : Train network

In [16]:
def train_network(network, train, l_rate, n_epoch, n_outputs):
	for epoch in range(n_epoch):
		sum_error = 0
		for row in train:
			outputs = forward_propagate(network, row)
			expected = [0 for i in range(n_outputs)]
			expected[row[-1]] = 1
			sum_error += sum([(expected[i]-outputs[i])**2 for i in range(len(expected))])
			backward_propagate_error(network, expected)
			update_weights(network, row, l_rate)
		print('> epoch=%d, lrate=%.3f, error=%.3f' % (epoch, l_rate, sum_error))

#### Exercise 12 : Define a dataset and test

In [17]:
dataset = [[2.7810836,2.550537003,0],
	[1.465489372,2.362125076,0],
	[3.396561688,4.400293529,0],
	[1.38807019,1.850220317,0],
	[3.06407232,3.005305973,0],
	[7.627531214,2.759262235,1],
	[5.332441248,2.088626775,1],
	[6.922596716,1.77106367,1],
	[8.675418651,-0.242068655,1],
	[7.673756466,3.508563011,1]]

In [18]:
seed(1)
n_inputs = len(dataset[0]) - 1
n_outputs = len(set([row[-1] for row in dataset]))
network = init_network(n_inputs, 2, n_outputs)
train_network(network, dataset, 0.5, 20, n_outputs)
for layer in network:
	print(layer)

> epoch=0, lrate=0.500, error=6.350
> epoch=1, lrate=0.500, error=5.531
> epoch=2, lrate=0.500, error=5.221
> epoch=3, lrate=0.500, error=4.951
> epoch=4, lrate=0.500, error=4.519
> epoch=5, lrate=0.500, error=4.173
> epoch=6, lrate=0.500, error=3.835
> epoch=7, lrate=0.500, error=3.506
> epoch=8, lrate=0.500, error=3.192
> epoch=9, lrate=0.500, error=2.898
> epoch=10, lrate=0.500, error=2.626
> epoch=11, lrate=0.500, error=2.377
> epoch=12, lrate=0.500, error=2.153
> epoch=13, lrate=0.500, error=1.953
> epoch=14, lrate=0.500, error=1.774
> epoch=15, lrate=0.500, error=1.614
> epoch=16, lrate=0.500, error=1.472
> epoch=17, lrate=0.500, error=1.346
> epoch=18, lrate=0.500, error=1.233
> epoch=19, lrate=0.500, error=1.132
[{'weights': [-1.4688375095432327, 1.850887325439514, 1.0858178629550297], 'output': 0.029980305604426185, 'delta': -0.0059546604162323625}, {'weights': [0.37711098142462157, -0.0625909894552989, 0.2765123702642716], 'output': 0.9456229000211323, 'delta': 0.002627965285

#### Exercise 13 : Predict

In [19]:
def predict(network, row):
	outputs = forward_propagate(network, row)
	return outputs.index(max(outputs))

#### Exercise 14 : Test the prediction

In [20]:
for row in dataset:
	prediction = predict(network, row)
	print('Expected=%d, Got=%d' % (row[-1], prediction))

Expected=0, Got=0
Expected=0, Got=0
Expected=0, Got=0
Expected=0, Got=0
Expected=0, Got=0
Expected=1, Got=1
Expected=1, Got=1
Expected=1, Got=1
Expected=1, Got=1
Expected=1, Got=1


#### Exercise 15 and 16 : Import dataset

In [21]:
def load_csv(filename):
	dataset = list()
	with open(filename, 'r') as file:
		csv_reader = reader(file)
		for row in csv_reader:
			if not row:
				continue
			dataset.append(row)
	return dataset

In [28]:
# Convert string column to float
def str_column_to_float(dataset, column):
	for row in dataset:
		row[column] = float(row[column].strip())
 
# Convert string column to integer
def str_column_to_int(dataset, column):
	class_values = [row[column] for row in dataset]
	unique = set(class_values)
	lookup = dict()
	for i, value in enumerate(unique):
		lookup[value] = i
	for row in dataset:
		row[column] = lookup[row[column]]
	return lookup

In [30]:
# Split a dataset into k folds
def cross_validation_split(dataset, n_folds):
	dataset_split = list()
	dataset_copy = list(dataset)
	fold_size = int(len(dataset) / n_folds)
	for i in range(n_folds):
		fold = list()
		while len(fold) < fold_size:
			index = randrange(len(dataset_copy))
			fold.append(dataset_copy.pop(index))
		dataset_split.append(fold)
	return dataset_split

# Calculate accuracy percentage
def accuracy_metric(actual, predicted):
	correct = 0
	for i in range(len(actual)):
		if actual[i] == predicted[i]:
			correct += 1
	return correct / float(len(actual)) * 100.0

# Evaluate an algorithm using a cross validation split
def evaluate_algorithm(dataset, algorithm, n_folds, *args):
	folds = cross_validation_split(dataset, n_folds)
	scores = list()
	for fold in folds:
		train_set = list(folds)
		train_set.remove(fold)
		train_set = sum(train_set, [])
		test_set = list()
		for row in fold:
			row_copy = list(row)
			test_set.append(row_copy)
			row_copy[-1] = None
		predicted = algorithm(train_set, test_set, *args)
		actual = [row[-1] for row in fold]
		accuracy = accuracy_metric(actual, predicted)
		scores.append(accuracy)
	return scores


In [32]:
def back_propagation(train, test, l_rate, n_epoch, n_hidden):
	n_inputs = len(train[0]) - 1
	n_outputs = len(set([row[-1] for row in train]))
	network = init_network(n_inputs, n_hidden, n_outputs)
	train_network(network, train, l_rate, n_epoch, n_outputs)
	predictions = list()
	for row in test:
		prediction = predict(network, row)
		predictions.append(prediction)
	return(predictions)

In [33]:
seed(1)

dataset = load_csv('seeds_dataset.csv')

for i in range(len(dataset[0])-1):
	str_column_to_float(dataset, i)

str_column_to_int(dataset, len(dataset[0])-1)


NameError: name 'reader' is not defined

#### Exercise 17 : Shuffle dataset and normalize columns

In [36]:
def dataset_minmax(dataset):
	minmax = list()
	stats = [[min(column), max(column)] for column in zip(*dataset)]
	return stats

def normalize_dataset(dataset, minmax):
	for row in dataset:
		for i in range(len(row)-1):
			row[i] = (row[i] - minmax[i][0]) / (minmax[i][1] - minmax[i][0])

In [34]:
minmax = dataset_minmax(dataset)
normalize_dataset(dataset, minmax)

In [35]:
n_folds = 5
l_rate = 0.3
n_epoch = 500
n_hidden = 5
scores = evaluate_algorithm(dataset, back_propagation, n_folds, l_rate, n_epoch, n_hidden)
print('Scores: %s' % scores)
print('Mean Accuracy: %.3f%%' % (sum(scores)/float(len(scores))))

NameError: name 'randrange' is not defined

#### Exercise 18 : Initialize network

#### Exercise 19 : Predict the test set on the trained network